## Example API Colav
Portal (alpha): http://clustercien.udea.edu.co:8889/ 
## Videos 
* [Citas Google Scholar](https://drive.google.com/file/d/16KGEHGMCyzHejyh_JIPYgyhzkgSTMxTC/view?usp=sharing)
  * [links](https://drive.google.com/file/d/1rLYtO3p7a69NovcNBJt9AJliS8cqd9w5/view?usp=sharing)
* [DevOps](https://drive.google.com/file/d/1hM_hcPu0pSq7KLpunXxEnkPpwHNDWQd0/view?usp=sharing)
  * [links](https://drive.google.com/file/d/1g0MncnWaBI2MkBQM8vNpu33n9gCkiPcj/view?usp=sharing)
* [Elasticsearch](https://drive.google.com/file/d/17wqExlzGmF4zMUcCTbLSH-vBbfuBSOqn/view?usp=sharing)
* [MongoDB](https://drive.google.com/file/d/1FbLKiWGDUQij1yzkw-wMNMWCds0u-kTg/view?usp=sharing)
  * [Notebook](https://drive.google.com/file/d/1IrRml7TNzYHbqetOeX3H3Kf9sjQxNiFA/view?usp=sharing)
* [Endpoint](https://drive.google.com/file/d/1nO8Ypz-U7S2yb2guMDvDtCYyN180u_Nj/view?usp=sharing)
* Capacitación API: https://drive.google.com/file/d/1y4-cm2tPLHPvDuNdqRwx6kQ8dztZfBIg/view?usp=sharing
  
## Dev code:
* https://github.com/muzgash/HunabKu/tree/fase3/hunabku/plugins
## Documentation
* http://clustercien.udea.edu.co:8888/apidoc/index.html

In [1]:
import pandas as pd
import getpass
import time 

In [2]:
key=getpass.getpass('API KEY')

API KEY ·········


TEST

In [3]:
max_docs=1
p=1
df=pd.read_json(f'http://clustercien.udea.edu.co:8888/api/faculties?apikey={key}&data=production&max=1&page=1')

Get the full documents

In [5]:
#Get total results
max_docs=1
p=1
tmp=pd.read_json('http://clustercien.udea.edu.co:8888/api/faculties?apikey={}&data=production&max={}&page={}'.format(key,max_docs,p))
#Get all documents
max_docs=100
pages=tmp['total_results'].loc[0]//max_docs

#Get all documents
max_docs=100
pages=tmp['total_results'].loc[0]//max_docs
if pages%max_docs>0:
    pages=pages+1
if pages>=1:
    df=pd.DataFrame()
    for p in range(1,pages+1):
        if p%10==0: print('+',end='')
        try:
            tmp=pd.read_json('http://clustercien.udea.edu.co:8888/api/faculties?apikey={}&data=production&max={}&page={}'.format(key,max_docs,p))
        except:
            tmp=pd.DataFrame()
        df=df.append( tmp['data'].values.tolist(),ignore_index=True  )
        time.sleep(1)
print()

++++++++++++++++++++++++++++++++


In [6]:
SAVE=False
if SAVE:
    df.to_json('udea.json',orient='records')

In [52]:
df=pd.read_json('https://github.com/restrepo/colav-api/raw/main/udea.json.gz')

In [53]:
df.shape

(25961, 31)

By document type

In [48]:
def get_affiliation(l,name= 'Universidad de Antioquia'):
    aff=False
    for d in l:
        if isinstance( d.get('affiliations'), list):
            for dd in d.get('affiliations'):
                if dd.get('name')==name:
                    aff=True
    return aff

In [49]:
df=df[df.authors.apply(lambda l: get_affiliation(l,name= 'Universidad de Antioquia'))].reset_index(drop=True)

In [50]:
df.shape

(25961, 31)

In [51]:
#df.to_json('udea.json',orient='records')

In [8]:
df['publication_type'].unique()

array(['book chapter', 'journal article',
       'conference proceedings article', 'article', 'preprint',
       'conference proceedings', 'series', 'review', 'book',
       'article; proceedings paper', 'editorial material',
       'journal issue', 'article; early access', 'letter',
       'article; data paper', 'journal', 'report', 'correction',
       'proceedings paper', 'dissertation', 'book review', '',
       'clinical trial', 'meeting abstract', 'biographical-item', 'note'],
      dtype=object)

In [9]:
nttmp=df[df['publication_type']==''].reset_index(drop=True)
nt=nttmp[nttmp.bibtex.str.split("@article{").str[-1].str.split('\n').str[3].str.contains('journal=').fillna(False)]
nnt=nttmp[~nttmp.bibtex.str.split("@article{").str[-1].str.split('\n').str[3].str.contains('journal=').fillna(False)]

Margen de error: Ver https://github.com/colav/Kahi/issues/31

In [10]:
nnt.bibtex.str.split("@article{").str[-1].str.split('\n').str[3:]

4       [  year={2009},,   publisher={Universidad de A...
10      [  year={2015},,   publisher={Universidad de A...
20      [  booktitle={Congresos CLABES},,   year={2015...
27      [  year={2005},,   publisher={Universidad de A...
29      [  year={2010},,   publisher={Universidad de A...
                              ...                        
7169                                                   []
7170                                                   []
7172                                                   []
7177    [  year={2004},,   publisher={Universidad de A...
7182                                 [  year={1992}, }, ]
Name: bibtex, Length: 950, dtype: object

In [11]:
nttmp.shape[0],nt.shape[0]

(7185, 6235)

In [12]:
df=df[( ( df['publication_type']!='preprint' ) & 
    ( df['publication_type']!='correction' ) &
    ( df['publication_type']!='book review' ) &
    ( df['publication_type']!= 'meeting abstract' ) &
    ( df['publication_type']!= 'biographical-item' ) &
    ( df['publication_type']!= 'note' ) 
   )].reset_index(drop=True)

In [13]:
df.shape

(25908, 31)

In [14]:
df['publication_type'].unique()

array(['book chapter', 'journal article',
       'conference proceedings article', 'article',
       'conference proceedings', 'series', 'review', 'book',
       'article; proceedings paper', 'editorial material',
       'journal issue', 'article; early access', 'letter',
       'article; data paper', 'journal', 'report', 'proceedings paper',
       'dissertation', '', 'clinical trial'], dtype=object)

In [15]:
df[df['year_published']>2014].shape

(9527, 31)

In [16]:
df[df['year_published']>2015].groupby('year_published')['year_published'].count()

year_published
2016    2042
2017    1835
2018    1676
2019    1594
2020     458
Name: year_published, dtype: int64

Comparisssion with https://link.lens.org/WsuTK1cXrbk for all publication types
```
2016 2043
2017 1844
2018 1742
2019 1706
2020 1723
TOTAL 29736
```
For Journal Article, Conference Proceedings (article), Book chapter, Book
```
2016 1534
2017 1479
2018 1462
2019 1487
2020 1593
TOTAL 20425 # https://link.lens.org/3StGzoZb8je
```

In [17]:
def get_db(l,dbs=['scopus','wos']):
    chkdb=False
    for d in l:
        if d.get('source') in dbs:
            chkdb=True
    return chkdb

In [18]:
dfs=df[df['external_ids'].apply(lambda l: get_db(l,dbs=['scopus','wos']) )].reset_index(drop=True)

In [19]:
dfs[dfs['year_published']>2015].groupby('year_published')['year_published'].count()

year_published
2016     987
2017    1057
2018    1100
2019    1127
2020     345
Name: year_published, dtype: int64

In [21]:
dfs.shape

(11432, 31)

In [39]:
def extract_corresponding_affiliation(l,affiliation='Universidad de Antioquia'):
    CORR=False
    for d in l:        
        if d.get('corresponding') and affiliation in [dd.get('name') for dd in d.get('affiliations')]:
            CORR=True
    return CORR

In [44]:
dfsa=dfs[dfs.authors.apply(lambda l: extract_corresponding_affiliation(l,affiliation='Universidad de Antioquia'))].reset_index(drop=True)
dfsa.shape

(2044, 31)

In [45]:
dfsa[dfsa['year_published']>2015].groupby('year_published')['year_published'].count()

year_published
2016    170
2017    188
2018    199
2019    213
2020     62
Name: year_published, dtype: int64

In [43]:
df[df.authors.apply(lambda l: extract_corresponding_affiliation(l,affiliation='Universidad de Antioquia'))].shape

(5442, 31)

In [7]:
from unidecode import unidecode

search for some author

In [8]:
oz=df[df.authors.apply(lambda l: True in [unidecode(d.get('full_name'))=='Oscar Alberto Zapata Norena' for d in l ] )]
oz.shape[0],oz[oz['topics'].astype(str).str.lower().str.contains('physics')].shape[0]

(29, 20)

Define a function to get a flat DataFrame with articles for an author

In [9]:
from unidecode import unidecode
def get_author(df,author):
    da=df[df.authors.apply(lambda l: True in [unidecode(d.get('full_name'))==unidecode(author) for d in l ] )].reset_index(drop=True)

    da['authors']=da.authors.apply(lambda l: [d.get('full_name') for d in l])

    da['doi']=da['external_ids'].apply(lambda l: [d.get('id') for d in l if d.get('source')=='doi']).str[0].fillna('')
    da['title']=da['titles'].apply(lambda l: [d.get('title') for d in l]).str[0]
    da['journal']=da['source'].apply(lambda d: d.get('title') )
    da['issn']=da['source'].apply(lambda d: [x.get('value') for x in d.get('serials')]).str[0]

    return da[['authors','title','publication_type','journal','issn',
               'year_published','issue', 'start_page',
               'end_page','doi','citations_count']]

In [10]:
authors=['Liliana Mahecha Ledesma',
'Joaquín Angulo Arizala',
'Sandra Lucía Posada Ochoa',
'Jaime Ricardo Rosero Noguera',
'Jorge Hernán Agudelo Trujillo',
'Marisol Medina Sierra',
'Mónica María Estrada Pareja',
'Sara María Márquez Girón',
'Mario Fernando Cerón Muñoz',
'Luis Fernando Galeano Vasco',
'Henry Cardona Cadavid',
'Zulma Tatiana Ruiz Cortez',
'Holmes Hernan Rodríguez Espinosa',
'José Fernando Guarín Montoya',
'Julio Echeverri Gómez',
'Elkin Mauricio Arboleda Zapata',
'Juan David Rodas González',
'Martha Eufemia Olivera Ángel',
'Jenny Jovana Chaparro Gutiérrez']

In [11]:
len(authors)

19

In [12]:
das=pd.DataFrame()
for a in authors:
    da=get_author(df,a)
    if da.empty:
        print(f'WARNING: missing {a}')
    da['autor_maestría']=a
    das=das.append(da).reset_index(drop=True)

In [13]:
das=das.reset_index(drop=True)[['autor_maestría','authors','title','publication_type',
                                'journal','issn','year_published','issue', 'start_page',
                                'end_page','doi','citations_count']]

Check data quality

In [14]:
das.shape

(869, 12)

In [15]:
das=das.drop_duplicates('title').reset_index(drop=True)

In [16]:
das.shape

(700, 12)

In [200]:
dasy=das[das['journal']!=''].reset_index(drop=True)
dasn=das[das['journal']==''].reset_index(drop=True)

In [201]:
dasy.shape[0]+dasn.shape[0]

700

In [202]:
dasy.shape

(582, 12)

In [203]:
dasy=dasy.drop_duplicates(subset=['journal','issn','year_published','issue','start_page','end_page','doi']).reset_index(drop=True)

In [204]:
das=dasy.append(dasn).reset_index(drop=True)

In [205]:
das.to_excel('produccion_total_maestria_en_ciencias_animales.xlsx',index=False)

In [206]:
das[:5]

,autor_maestría,authors,title,publication_type,journal,issn,year_published,issue,start_page,end_page,doi,citations_count
0,Liliana Mahecha Ledesma,"[Joaquin Angulo Arizala, Luis Miguel Gómez, Liliana Mahecha Ledesma, Estefanía Mejía, Javier Henao, Carolina Mesa]","Calf’s sex, parity and the hour of harvest after calving affect colostrum quality of dairy cows grazing under high tropical conditions",journal article,Tropical Animal Health and Production,00494747,2015,4,699,705,10.1007/s11250-015-0781-z,13
1,Liliana Mahecha Ledesma,"[Juan Londoño C, Liliana Mahecha Ledesma, Joaquín Angulo A]",Desempeño agronómico y valor nutritivo de Tithonia diversifolia (Hemsl.) A Gray para la alimentación de bovinos,journal article,Revista Colombiana de Ciencia Animal Recia,20274297,2019,1,,,10.24188/recia.v0.n0.2019.693,5
2,Liliana Mahecha Ledesma,"[Joaquin Angulo Arizala, B. Hiller, E. Albrecht, Martha Eufemia Olivera Angel, Liliana Mahecha Ledesma, G. Nuernberg, D. Dannenberger, K. Nuernberg]",Effect of different dietary fats on protein expression of sterol regulatory element-binding protein 1 (SREBP-1) in mammary gland tissue of lactating cows,article,LIVESTOCK SCIENCE,18711413,2012,2-3,300,304,10.1016/j.livsci.2011.10.008,9
3,Liliana Mahecha Ledesma,"[Esperanza Prieto-Manrique, Julio Ernesto Vargas-Sánchez, Joaquin Angulo Arizala, Liliana Mahecha Ledesma]",Suplementación con aceite de girasol sobre ácidos grasos de la leche en una lechería tropical,journal article,Revista Colombiana de Ciencia Animal Recia,20274297,2016,,297,309,10.24188/recia.v8.n0.2016.385,1
4,Liliana Mahecha Ledesma,"[Wilson Barragán Hernández, Liliana Mahecha Ledesma, Yasmín Socorro Cajas-Girón]","Efecto de sistemas silvopastoriles en la producción y composición de la leche bajo condiciones del valle medio del rio sinú, Colombia",journal article,Revista Colombiana de Ciencia Animal Recia,20274297,2016,2,187,196,10.24188/recia.v8.n2.2016.186,5


In [207]:
das[das['year_published']>2015].shape[0]

182

In [208]:
das[das['year_published']>2015].citations_count.sum()

575

In [209]:
das[das['year_published']>2015].citations_count.sum()/das[das['year_published']>2012].shape[0]

1.8370607028753994

In [210]:
das.citations_count.sum()/das.shape[0]

7.8514376996805115

In [211]:
503/582

0.8642611683848798

API Author

In [13]:
url=f'http://clustercien.udea.edu.co:8888/api/authors?apikey={key}&data=production&id=5fc66fdfb246cc0887190aa6&max=100&page=1'

In [14]:
import pandas as pd

In [15]:
import requests

In [16]:
r=requests.get(url)

In [19]:
pd.DataFrame(r.json()['data'])[:1]

,urls,source_checked,keywords,open_access_status,citations_count,topics,publication_type,end_page,author_count,date_published,...,volume,source,references,issue,year_published,bibtex,citations_link,languages,_id,references_count
0,[{'url': 'https://www.scopus.com/inward/record...,"[{'source': 'lens', 'ts': 1617579319}, {'sourc...",[],green,19,"[{'source': 'lens', 'topics': ['Particle physi...",journal article,,4,1471392000,...,94,"{'subjects': {}, 'institution_id': '', 'source...",[],3,2016,"@article{von2016radiative,\n title={Radiative...",/scholar?cites=8886486850903082704&as_sdt=2005...,[en],606a4d38aa884b9a156300f1,99


In [23]:
author='Cesar Augusto Barrero Meneses'
da=df[df.authors.apply(lambda l: True in [unidecode(d.get('full_name'))==unidecode(author) for d in l ] )].reset_index(drop=True)